# mberta for polarozation classification

---



## Introduction

In this part of the starter notebook, we will take you through the process of all three Subtasks.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Subtask 1 - Polarization detection

This is a binary classification to determine whether a post contains polarized content (Polarized or Not Polarized).

In [2]:
!unzip /content/drive/MyDrive/dev_phase.zip

Archive:  /content/drive/MyDrive/dev_phase.zip
   creating: subtask1/
   creating: subtask1/dev/
  inflating: subtask1/dev/nep.csv    
  inflating: subtask1/dev/ita.csv    
  inflating: subtask1/dev/hin.csv    
  inflating: subtask1/dev/hau.csv    
  inflating: subtask1/dev/spa.csv    
  inflating: subtask1/dev/deu.csv    
  inflating: subtask1/dev/fas.csv    
  inflating: subtask1/dev/arb.csv    
  inflating: subtask1/dev/amh.csv    
  inflating: subtask1/dev/tur.csv    
  inflating: subtask1/dev/zho.csv    
  inflating: subtask1/dev/eng.csv    
  inflating: subtask1/dev/urd.csv    
   creating: subtask1/train/
  inflating: subtask1/train/nep.csv  
  inflating: subtask1/train/ita.csv  
  inflating: subtask1/train/hin.csv  
  inflating: subtask1/train/fas.csv  
  inflating: subtask1/train/deu.csv  
  inflating: subtask1/train/hau.csv  
  inflating: subtask1/train/spa.csv  
  inflating: subtask1/train/arb.csv  
  inflating: subtask1/train/tur.csv  
  inflating: subtask1/train/zho.csv  


## Imports

In [3]:
import pandas as pd

from sklearn.metrics import recall_score, precision_score, f1_score
import numpy as np

import torch

from sklearn.metrics import f1_score

from transformers import (
    AutoTokenizer,
    AutoConfig,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding
)
from torch.utils.data import Dataset

In [4]:
!pip install wandb

In [5]:
import wandb

# Disable wandb logging for this script
wandb.init(mode="disabled")

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


Using mberta model for classification


In [7]:

drive.mount('/content/drive')
import torch
torch.cuda.empty_cache()
import pandas as pd
import numpy as np
import torch
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding
)
from torch.utils.data import Dataset


# Dataset class

class PolarizationDataset(Dataset):
    def __init__(self, df, tokenizer, require_labels=True):
        self.texts = df["text"].fillna("").tolist()
        if require_labels:
            self.labels = df["polarization"].astype(int).tolist()
        else:
            self.labels = [0] * len(self.texts)  # dummy labels
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        enc = self.tokenizer(
            self.texts[idx],
            truncation=True,
            padding="max_length",
            max_length=128,
            return_tensors="pt"
        )
        item = {k: v.squeeze(0) for k,v in enc.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item


# Load data

languages = ["eng","hin","spa","urd","zho","arb"]
data = {}

for lang in languages:
    train_df = pd.read_csv(f"subtask1/train/{lang}.csv")   # labeled
    dev_df   = pd.read_csv(f"subtask1/dev/{lang}.csv")     # unlabeled
    data[lang] = {"train": train_df, "dev": dev_df}

tokenizer = AutoTokenizer.from_pretrained("microsoft/mdeberta-v3-base")


# Metric

def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    return {"f1_macro": f1_score(p.label_ids, preds, average="macro")}


# MAIN LOOP: TRAIN/VAL SPLIT + DEV PREDICTION

f1_results = []
predicted_outputs = {}

for lang, dfs in data.items():
    print("\n====================================")
    print(f"LANGUAGE: {lang}")

    train_df = dfs["train"]
    dev_df   = dfs["dev"]

    # Filter ONLY labeled training rows
    train_labeled = train_df.dropna(subset=["polarization"]).reset_index(drop=True)

    # Split train into train/validation
    train_split, val_split = train_test_split(
        train_labeled,
        test_size=0.20,
        stratify=train_labeled["polarization"],
        random_state=42,
        shuffle=True,
    )

    print(f"Train size: {len(train_split)},  Validation size: {len(val_split)}")

    train_dataset = PolarizationDataset(train_split, tokenizer, require_labels=True)
    val_dataset   = PolarizationDataset(val_split,   tokenizer, require_labels=True)

    # Train model
    model = AutoModelForSequenceClassification.from_pretrained(
        "microsoft/mdeberta-v3-base", num_labels=2
    )

    training_args = TrainingArguments(
        output_dir=f"./model_{lang}",
        learning_rate=2e-5,
        num_train_epochs=30,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        eval_strategy="epoch",
        save_strategy="no",
        logging_steps=20
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        compute_metrics=compute_metrics,
        data_collator=DataCollatorWithPadding(tokenizer)
    )

    trainer.train()

    # Compute F1 on validation
    metrics = trainer.evaluate()
    f1 = metrics["eval_f1_macro"]
    print(f"{lang} Validation F1 = {f1:.4f}")

    f1_results.append({"language": lang, "f1_macro": f1})

    # Predict on dev (UNLABELED)
    print(f"🔮 Predicting for dev set ({len(dev_df)} rows)...")
    dev_dataset = PolarizationDataset(dev_df, tokenizer, require_labels=False)
    preds = trainer.predict(dev_dataset)
    pred_labels = np.argmax(preds.predictions, axis=1)

    dev_df["predicted_polarization"] = pred_labels
    predicted_outputs[lang] = dev_df


# SAVE PREDICTIONS

for lang, df_pred in predicted_outputs.items():
    df_pred.to_csv(f"{lang}_dev_predicted.csv", index=False)
    print(f"Saved: {lang}_dev_predicted.csv")


# FINAL F1 SCORES

f1_df = pd.DataFrame(f1_results)
print("\nFINAL F1 SCORES:")
print(f1_df)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


spm.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(



LANGUAGE: eng
Train size: 2140,  Validation size: 536


pytorch_model.bin:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

...

zho Validation F1 = 0.8808
🔮 Predicting for dev set (214 rows)...

LANGUAGE: arb
Train size: 2704,  Validation size: 676


Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/mdeberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,F1 Macro
1,0.463700,0.443797,0.773060
2,0.376000,0.475424,0.792899
3,0.261700,0.546623,0.788216
4,0.170800,0.826366,0.788383
5,0.133600,1.120317,0.766447
6,0.092800,1.086063,0.791143
7,0.069500,1.301000,0.790573
8,0.036400,1.471808,0.789203
9,0.059800,1.564668,0.781683
10,0.014400,1.709036,0.760380


Epoch,Training Loss,Validation Loss,F1 Macro
1,0.463700,0.443797,0.773060
2,0.376000,0.475424,0.792899
3,0.261700,0.546623,0.788216
4,0.170800,0.826366,0.788383
5,0.133600,1.120317,0.766447
6,0.092800,1.086063,0.791143
7,0.069500,1.301000,0.790573
8,0.036400,1.471808,0.789203
9,0.059800,1.564668,0.781683
10,0.014400,1.709036,0.760380


arb Validation F1 = 0.7696
🔮 Predicting for dev set (169 rows)...
Saved: eng_dev_predicted.csv
Saved: hin_dev_predicted.csv
Saved: spa_dev_predicted.csv
Saved: urd_dev_predicted.csv
Saved: zho_dev_predicted.csv
Saved: arb_dev_predicted.csv

FINAL F1 SCORES:
  language  f1_macro
0      eng  0.787699
1      hin  0.803049
2      spa  0.747121
3      urd  0.733837
4      zho  0.880776
5      arb  0.769575
